# CAFEPP (utilising cafepp_daily(.py))

***

## This example will produce a set of daily SST files across the V1 assimilation run (interpolated onto a 1x1 latxlon grid), and upon success, generate an average over the nino3.4 region and plot the time-series.

***

# Various settings required:

## BATCH determines whether it will be sent to the queue via qsub command or run interactively.

## CLEAN determines whether the run directory will be emptied prior to processing.

In [1]:
print('BEGIN')

from __future__ import print_function #this is to allow print(,file=xxx) feature

import datetime
import shutil
import os

#dvar='tos' #variable to generate as well as further proces and plot.
#dvar='ts'
#dvar='tas'
#dvar='thetao'
#dvar='psl'
#dvar='pr'
dvar='dummy'

BATCH=True #submit to queue
BATCH=False #run interactively but in a batch temporary area.

CLEAN=False #don't remove rundir, just use it.
CLEAN=True #remove rundir and recreate it.

if(BATCH):
  print('Submitting to queue.')

if(CLEAN):
  print('Removing run directory and reestablish it.')

predir='/short/v14/mac599'
topdir=predir+'/'+'cafepp'
script='cafepp_daily_assimilation_year_month.py'
#script='cafepp_daily_assimilation.py'
#script='cafepp_daily_control.py'
#script='cafepp_daily_forecast.py'

rundir=topdir+'/'+'rundir20180601103944'
#rundir=topdir+'/'+'rundir'+datetime.datetime.today().strftime('%Y%m%d%H%M%S')
#rundir=topdir+'/'+'rundir' #temporary running directory

print('Running in directory '+rundir)
print('Using script '+script)

srcdir='/home/599/mac599/decadal' #main cafepp code
prodir=srcdir+'/paper_analysis' #project directory

print('END')

BEGIN
Removing run directory and reestablish it.
Running in directory /short/v14/mac599/cafepp/rundir20180601103944
Using script cafepp_daily_assimilation_year_month.py
END


# Here we make directories, copy across necessary JSON and python code.

In [2]:
print('BEGIN')

print('copying json files, generating symlinks, cmor tables, queue script, if necessary.')
if(os.path.exists(rundir) and CLEAN):
  shutil.rmtree(rundir)
  os.mkdir(rundir)
  os.mkdir(rundir+'/'+'JsonTemplates') # exist_ok=True only python3
elif(not os.path.exists(rundir)):
  os.mkdir(rundir)
  os.mkdir(rundir+'/'+'JsonTemplates') # exist_ok=True only python3

os.symlink(predir+'/'+'CMIP6',rundir+'/'+'CMIP6')
os.symlink(prodir+'/'+'TablesTemplates',rundir+'/'+'TablesTemplates')
os.symlink(prodir+'/'+'cmip6-cmor-tables',rundir+'/'+'cmip6-cmor-tables')

if(BATCH):
  ifh=open(prodir+'/'+'qjob.csh')
  ofh=open(rundir+'/'+'qjob.csh','w')
  for i,line in enumerate(ifh):
    line=line.replace('RUNDIR',rundir)
    print(line,file=ofh,end='')
  ifh.close()
  ofh.close()

vector_string=['decadal_diag.py','cafepp_daily.py','app_funcs.py']
for i,file_now in enumerate(vector_string):
  print('Copying '+srcdir+'/'+file_now+' to ',rundir)
  shutil.copyfile(srcdir+'/'+file_now,rundir+'/'+file_now)

#vector_string=[script]
#for i,file_now in enumerate(vector_string):
#  print('Copying '+prodir+'/'+file_now+' to ',rundir)
#  shutil.copyfile(prodir+'/'+file_now,rundir+'/'+file_now)
  
vector_string=[]
vector_string.append(script) #may need to do edits?
vector_string.append('cafepp_daily_assimilation.json')
#vector_string.append('cafepp_daily_forecast_experiments.json')
#vector_string.append('JsonTemplates'+'/'+'cafepp_experiments.json')
vector_string.append('JsonTemplates'+'/'+'cafepp_csiro-gfdl.json')
vector_string.append('JsonTemplates'+'/'+'cafepp_vars.json')
for i,file_now in enumerate(vector_string):
  print('Copying '+prodir+'/'+file_now+' to ',rundir+'/'+file_now)
  shutil.copyfile(prodir+'/'+file_now,rundir+'/'+file_now)
  
vector_string=[]
vector_string.append('cafepp_experiments.json')
for i,file_now in enumerate(vector_string):
  print('Copying '+prodir+'/JsonTemplates/'+file_now+' to '+rundir+'/'+file_now)
  shutil.copyfile(prodir+'/JsonTemplates/'+file_now,rundir+'/'+file_now)
  
print('Processing cafepp_daily_assimilation.json')
ifh=open(prodir+'/'+'cafepp_daily_assimilation.json')
ofh=open(rundir+'/'+'cafepp_daily_assimilation.json','w')
for i,line in enumerate(ifh):
  token1=[str(x) for x in line.split(':')]
  token2=(token1[0].replace(' ',''))
  token3=(token2.replace('"',''))
  if(token3=='dvar'):
    line='     "dvar":"'+dvar+'",\n'
#  elif(token3=='grid_label'):
#    line='     "grid_label":"gr2",\n'
#  elif(token3=='grid'):
#    line='     "grid":"data regridded via linear interpolation to a 1x1 degree latxlon  (180x360)grid from the native 300x360 latxlon tri-polar grid",\n'
  print(line,file=ofh,end='')
ifh.close()
ofh.close()

#break
print('END')

BEGIN
copying json files, generating symlinks, cmor tables, queue script, if necessary.
Copying /home/599/mac599/decadal/decadal_diag.py to  /short/v14/mac599/cafepp/rundir20180601103944
Copying /home/599/mac599/decadal/cafepp_daily.py to  /short/v14/mac599/cafepp/rundir20180601103944
Copying /home/599/mac599/decadal/app_funcs.py to  /short/v14/mac599/cafepp/rundir20180601103944
Copying /home/599/mac599/decadal/paper_analysis/cafepp_daily_assimilation_year_month.py to  /short/v14/mac599/cafepp/rundir20180601103944/cafepp_daily_assimilation_year_month.py
Copying /home/599/mac599/decadal/paper_analysis/cafepp_daily_assimilation.json to  /short/v14/mac599/cafepp/rundir20180601103944/cafepp_daily_assimilation.json
Copying /home/599/mac599/decadal/paper_analysis/JsonTemplates/cafepp_csiro-gfdl.json to  /short/v14/mac599/cafepp/rundir20180601103944/JsonTemplates/cafepp_csiro-gfdl.json
Copying /home/599/mac599/decadal/paper_analysis/JsonTemplates/cafepp_vars.json to  /short/v14/mac599/cafepp/

# We establish where to execute the job.

## We import a function that is relatively simple to loop over all necessary years, months, ensembles as required.
## Different applications will require a different module to be written (often small and relatively simple).

In [3]:
print('BEGIN')

os.chdir(rundir)
print('Current Working Directory=',os.getcwd())

import getpass
import numpy as np
import numpy.ma as ma
import os
from time import strftime
import netCDF4
from math import radians, cos, sin, asin, sqrt
import seawater
#import sys
import getopt
import string
from decadal_diag import MustHaveAllLevs,diag_acc_drake,diag_acc_africa,diag_mozmbq,diag_aabw,diag_nadw,\
diag_pp,diag_nflux,diag_ep,diag_ssh,diag_moc,diag_moc_atlantic,diag_moc_pacific,diag_moc_indian,\
diag_shice_cover,diag_nhice_cover,diag_nino34,xtra_nino34,init_data,sum_data,avg_data,filemonth_index,\
data_wavg,time_avg,diag_nhblocking_index,diag_rws5,finish,diag_msftyz,make_mask3D,diag_mfo,transPort,\
diag_rws500,create_odirs,create_ofils,diag_iod,diag_iod,xtra_iod,atmos_vertical_interpolate,diag_isothetaoNc,\
calc_iso_surface,calc_isoN,grab_var_meta,diag_psl,diag_hfls,diag_heat_content,diag_salt_content,\
diag_north_heat_trans,diag_north_salt_trans,ocean_vertical_interpolate,diag_thetao0to80m,diag_varNl,\
uncomment_json,process_json,get_daily_indices_for_monthlyave,generate_daily_month_indices,diag_maxdTbydz,diag_depmaxdTbydz,\
diag_dTbydz,shade_2d_simple,shade_2d_latlon,diag_zmld_boyer,zmld_boyer,sigmatheta,diag_zmld_so,\
zmld_so,diag_spice,spice,diag_bigthetao,diag_soabs,diag_spiciness,diag_potrho,fractional_year_from_num2date,\
new_monthly_array_shape,restrict_input_files,get_timestamp_number

#from decadal_diag import generate_daily_month_indices

import cmor
import cdtime
from app_funcs import *
import json
import pprint
from datetime import date
import filecmp
from shutil import copyfile
import cdms2
import inspect
import socket
import glob
from matplotlib.mlab import griddata
import scipy.sparse as sps
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point
import matplotlib as mpl
mpl.rcParams['mathtext.default'] = 'regular'
import matplotlib.pyplot as plt
from gridfill import fill as poisson_fill

function_name=script.split('.py')[0]
print('function_name=',function_name)
import importlib

#import cafepp_daily_assimilation_year_month
#import eval(function_name)

cafepp_daily = importlib.import_module(function_name)

#print(dir(cafepp_daily_control))
print('END')

BEGIN
Current Working Directory= /short/v14/mac599/cafepp/rundir20180601103944
you have not specified the environment variable: 'CDAT_LOCATION' , trying to import cdms2 anyway
function_name= cafepp_daily_assimilation_year_month
END


# Either submit it to the queue or run interactively.

In [ ]:
print('BEGIN')

SWITCH_OFF=True #temporary for easy skipping of this section.
#SWITCH_OFF=False

if(not SWITCH_OFF):
  if(BATCH):
    os.chmod(script,500)
    os.system('qsub '+rundir+'/'+'qjob.csh')
  else:
    print('Current Working Directory=',os.getcwd())
    os.chdir(rundir)
    cafepp_daily.main(rundir)
print('END')

BEGIN
END


# Process daily assimilation output into one continuous file...

working but issue with too many open files.

In [ ]:
print('BEGIN')

SWITCH_OFF=True #temporary for easy skipping of this section.
SWITCH_OFF=False

if(not SWITCH_OFF):

  cafe_experiment='coupled_da/OUTPUT-2step-nobreeding-carbon2'
  ybeg_first=2002 #2002
  yend_last=2016 #2016
  ybeg=2009 #2002
  yend=2016 #2016
  mbeg_first=1 #2
  mend_last=6 #6
  mbeg_norm=1 #1
  mend_norm=12 #12
  
  dvar='tas'
  #dvar='pr'
  #dvar='tos'
  #dvar='psl'
  
  line_kwargs= \
    'rundir='+rundir+ \
    ' cafe_experiment='+cafe_experiment+ \
    ' dvar='+dvar+ \
    ' ybeg_first='+str(ybeg_first)+' yend_last='+str(yend_last)+ \
    ' ybeg='+str(ybeg)+' yend='+str(yend)+ \
    ' mbeg_first='+str(mbeg_first)+' mend_last='+str(mend_last)+ \
    ' mbeg_norm='+str(mbeg_norm)+' mend_norm='+str(mend_norm) #no space after first item ' is important

  if(BATCH):
    os.chmod(script,500)
    print('Copying and editing '+prodir+'/'+'qjob.csh to '+rundir+'/'+'qjob.csh')
    ifh=open(prodir+'/'+'qjob.csh')
    ofh=open(rundir+'/'+'qjob.csh','w')
    for i,line in enumerate(ifh):
      line=line.replace('CAFEPP_SCRIPT','./'+script+' '+line_kwargs)
      line=line.replace('RUNDIR',rundir)
      line=line.replace('CONDA_SOURCE','. /short/v14/mac599/anaconda3/etc/profile.d/conda.sh')
      line=line.replace('CONDA_ACTIVATE','conda activate cafepp_27_scipy')
      print(line,file=ofh,end='')
    ifh.close()
    ofh.close()
    os.system('qsub '+rundir+'/'+'qjob.csh')
  else:
    print('Current Working Directory=',os.getcwd())
    os.chdir(rundir)
    
    kwargs=dict(x.split('=', 1) for x in line_kwargs.split(' '))
    print('kwargs=',kwargs)
      
    cafepp_daily.main(**kwargs)
        
print('END')

BEGIN
Current Working Directory= /short/v14/mac599/cafepp/rundir20180601103944
kwargs= {'ybeg_first': '2002', 'mbeg_first': '1', 'yend': '2016', 'mbeg_norm': '1', 'mend_norm': '12', 'ybeg': '2009', 'rundir': '/short/v14/mac599/cafepp/rundir20180601103944', 'dvar': 'tas', 'cafe_experiment': 'coupled_da/OUTPUT-2step-nobreeding-carbon2', 'yend_last': '2016', 'mend_last': '6'}
Processing cafepp_daily_assimilation.json
Processing cafepp_experiments.json
MAIN
hostname= raijin6
Running cafepp from JSON instructions: cafepp.json
fh_printfile= <ipykernel.iostream.OutStream object at 0x7fa13baffc90>
Found required output experiment : coupled_da/OUTPUT-2step-nobreeding-carbon2
hostname= raijin
storage_machine_no1= tube-hba
['/short/v14/mac599/anaconda3/envs/cafepp_27_scipy/lib/python2.7/site-packages/ipykernel_launcher.py', '-f', '/home/599/mac599/.local/share/jupyter/runtime/kernel-d0230426-8b93-40e6-8029-e5bc385fd75c.json']
Found required output variable: tas
Output CMIP6 file: CMIP6/CMIP/CSIRO

Output CMIP6 file: CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r1i1p2f1/day/tas/gn/v20171025/tas_day_historical_CAFE-1-0_r1i1p2f1_gn_20020101-20161231.nc
cmip6-cmor-tables/Tables/CMIP6_day.json
Try
input_files= ['/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090201/atmos_daily_2009_02_01.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090201/atmos_daily_2009_02_02.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090201/atmos_daily_2009_02_03.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090201/atmos_daily_2009_02_04.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090201/atmos_daily_2009_02_05.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090201/atmos_daily_2009_02_06.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUT

Output CMIP6 file: CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r1i1p2f1/day/tas/gn/v20171025/tas_day_historical_CAFE-1-0_r1i1p2f1_gn_20020101-20161231.nc
cmip6-cmor-tables/Tables/CMIP6_day.json
Try
input_files= ['/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090301/atmos_daily_2009_03_01.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090301/atmos_daily_2009_03_02.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090301/atmos_daily_2009_03_03.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090301/atmos_daily_2009_03_04.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090301/atmos_daily_2009_03_05.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090301/atmos_daily_2009_03_06.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUT

Found required output variable: tas
Output CMIP6 file: CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r1i1p2f1/day/tas/gn/v20171025/tas_day_historical_CAFE-1-0_r1i1p2f1_gn_20020101-20161231.nc
cmip6-cmor-tables/Tables/CMIP6_day.json
Try
input_files= ['/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090401/atmos_daily_2009_04_01.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090401/atmos_daily_2009_04_02.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090401/atmos_daily_2009_04_03.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090401/atmos_daily_2009_04_04.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090401/atmos_daily_2009_04_05.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090401/atmos_daily_2009_04_06.nc', '/short/v14/tok599/co

Output CMIP6 file: CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r1i1p2f1/day/tas/gn/v20171025/tas_day_historical_CAFE-1-0_r1i1p2f1_gn_20020101-20161231.nc
cmip6-cmor-tables/Tables/CMIP6_day.json
Try
input_files= ['/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090501/atmos_daily_2009_05_01.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090501/atmos_daily_2009_05_02.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090501/atmos_daily_2009_05_03.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090501/atmos_daily_2009_05_04.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090501/atmos_daily_2009_05_05.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20090501/atmos_daily_2009_05_06.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUT

...

o= 0
['tas', 'day', 'historical', 'CAFE-1-0', 'r1i1p2f1', 'gn', '20130701-20130731.nc']
['20130701-20130731', '']
['20130701', '20130731']
ybeg,yend,mbeg,mend= 2013 2013 07 07
<type 'list'>
tas_day_historical_CAFE-1-0_r1i1p2f1_gn_2013-2013.nc
file_name:  CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r1i1p2f1/day/tas/gn/v20171025/tas_day_historical_CAFE-1-0_r1i1p2f1_gn_20130701-20130731.nc
odir:  CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r1i1p2f1/day/tas/gn/v20171025
ofil:  tas_day_historical_CAFE-1-0_r1i1p2f1_gn_20130701-20130731.nc
ofil_modified:  tas_day_historical_CAFE-1-0_r1i1p2f1_gn_20130701-20130731.nc
season:  None
broadcast:  dummy
Will need to put in "importance flag", perhaps it can go in another standard metadata tag?
Will need to move this CMIP6 file to slightly different name to clearly specify that it is a special season where the time axis is not continguous.
Output:  CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r1i1p2f1/day/tas/gn/v20171025/tas_day_historical_CAFE-1-0_r1i1p2f1_gn_20130701-20

Output CMIP6 file: CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r1i1p2f1/day/tas/gn/v20171025/tas_day_historical_CAFE-1-0_r1i1p2f1_gn_20020101-20161231.nc
cmip6-cmor-tables/Tables/CMIP6_day.json
Try
input_files= ['/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20130901/atmos_daily_2013_09_01.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20130901/atmos_daily_2013_09_02.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20130901/atmos_daily_2013_09_03.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20130901/atmos_daily_2013_09_04.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20130901/atmos_daily_2013_09_05.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20130901/atmos_daily_2013_09_06.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUT

Output CMIP6 file: CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r1i1p2f1/day/tas/gn/v20171025/tas_day_historical_CAFE-1-0_r1i1p2f1_gn_20020101-20161231.nc
cmip6-cmor-tables/Tables/CMIP6_day.json
Try
input_files= ['/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131001/atmos_daily_2013_10_01.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131001/atmos_daily_2013_10_02.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131001/atmos_daily_2013_10_03.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131001/atmos_daily_2013_10_04.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131001/atmos_daily_2013_10_05.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131001/atmos_daily_2013_10_06.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUT

Output CMIP6 file: CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r1i1p2f1/day/tas/gn/v20171025/tas_day_historical_CAFE-1-0_r1i1p2f1_gn_20020101-20161231.nc
cmip6-cmor-tables/Tables/CMIP6_day.json
Try
input_files= ['/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131101/atmos_daily_2013_11_01.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131101/atmos_daily_2013_11_02.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131101/atmos_daily_2013_11_03.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131101/atmos_daily_2013_11_04.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131101/atmos_daily_2013_11_05.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131101/atmos_daily_2013_11_06.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUT

Found required output variable: tas
Output CMIP6 file: CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r1i1p2f1/day/tas/gn/v20171025/tas_day_historical_CAFE-1-0_r1i1p2f1_gn_20020101-20161231.nc
cmip6-cmor-tables/Tables/CMIP6_day.json
Try
input_files= ['/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131201/atmos_daily_2013_12_01.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131201/atmos_daily_2013_12_02.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131201/atmos_daily_2013_12_03.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131201/atmos_daily_2013_12_04.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131201/atmos_daily_2013_12_05.nc', '/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20131201/atmos_daily_2013_12_06.nc', '/short/v14/tok599/co

# Process daily assimilation output into separate year/month files...

In [ ]:
print('BEGIN')

SWITCH_OFF=True #temporary for easy skipping of this section.
#SWITCH_OFF=False

if(not SWITCH_OFF):

  cafe_experiment='coupled_da/OUTPUT-2step-nobreeding-carbon2'
  ybeg_first=2002 #2002
  yend_last=2016 #2016
  ybeg=2002
  yend=2016
  mbeg_first=1 #2
  mend_last=6 #6
  mbeg_norm=1 #1
  mend_norm=12 #12
  
  line_kwargs= \
    'rundir='+rundir+ \
    ' cafe_experiment='+cafe_experiment+ \
    ' ybeg_first='+str(ybeg_first)+' yend_last='+str(yend_last)+ \
    ' ybeg='+str(ybeg)+' yend='+str(yend)+ \
    ' mbeg_first='+str(mbeg_first)+' mend_last='+str(mend_last)+ \
    ' mbeg_norm='+str(mbeg_norm)+' mend_norm='+str(mend_norm) #no space after first item ' is important

  if(BATCH):
    os.chmod(script,500)
    print('Copying and editing '+prodir+'/'+'qjob.csh to '+rundir+'/'+'qjob.csh')
    ifh=open(prodir+'/'+'qjob.csh')
    ofh=open(rundir+'/'+'qjob.csh','w')
    for i,line in enumerate(ifh):
      line=line.replace('CAFEPP_SCRIPT','./'+script+' '+line_kwargs)
      line=line.replace('RUNDIR',rundir)
      line=line.replace('CONDA_SOURCE','. /short/v14/mac599/anaconda3/etc/profile.d/conda.sh')
      line=line.replace('CONDA_ACTIVATE','conda activate cafepp_27_scipy')
      print(line,file=ofh,end='')
    ifh.close()
    ofh.close()
    os.system('qsub '+rundir+'/'+'qjob.csh')
  else:
    print('Current Working Directory=',os.getcwd())
    os.chdir(rundir)
    
    kwargs=dict(x.split('=', 1) for x in line_kwargs.split(' '))
    print('kwargs=',kwargs)
      
    cafepp_daily.main(**kwargs)
        
print('END')

# Process daily multi-ensemble forecast output...

In [ ]:
print('BEGIN')

SWITCH_OFF=True #temporary for easy skipping of this section.
#SWITCH_OFF=False

if(not SWITCH_OFF):

  cafe_experiment='v1_forecast'
  ybeg_first=2002 #2002
  yend_last=2016 #2016
  ybeg=2016
  yend=2016
  mbeg_first=1 #2
  mend_last=5 #5
  mbeg_norm=1 #1
  mend_norm=12 #12
  ebeg=1 #1
  eend=11 #11
  
  line_kwargs= \
    'rundir='+rundir+ \
    ' cafe_experiment='+cafe_experiment+ \
    ' ybeg_first='+str(ybeg_first)+' yend_last='+str(yend_last)+ \
    ' ybeg='+str(ybeg)+' yend='+str(yend)+ \
    ' mbeg_first='+str(mbeg_first)+' mend_last='+str(mend_last)+ \
    ' mbeg_norm='+str(mbeg_norm)+' mend_norm='+str(mend_norm)+ \
    ' ebeg='+str(ebeg)+' eend='+str(eend)  #no space after first item ' is important

  if(BATCH):
    os.chmod(script,500)
    print('Copying and editing '+prodir+'/'+'qjob.csh to '+rundir+'/'+'qjob.csh')
    ifh=open(prodir+'/'+'qjob.csh')
    ofh=open(rundir+'/'+'qjob.csh','w')
    for i,line in enumerate(ifh):
      line=line.replace('CAFEPP_SCRIPT','./'+script+' '+line_kwargs)
      line=line.replace('RUNDIR',rundir)
      line=line.replace('CONDA_SOURCE','. /short/v14/mac599/anaconda3/etc/profile.d/conda.sh')
      line=line.replace('CONDA_ACTIVATE','conda activate cafepp_27_scipy')
      print(line,file=ofh,end='')
    ifh.close()
    ofh.close()
    os.system('qsub '+rundir+'/'+'qjob.csh')
  else:
    print('Current Working Directory=',os.getcwd())
    os.chdir(rundir)
    
    kwargs=dict(x.split('=', 1) for x in line_kwargs.split(' '))
    print('kwargs=',kwargs)
      
    cafepp_daily.main(**kwargs)
        
print('END')

# Process daily control output...

In [ ]:
print('BEGIN')

SWITCH_OFF=True #temporary for easy skipping of this section.
SWITCH_OFF=False

if(not SWITCH_OFF):
  
  daily_data_layout='noleap_1fileperyear' #this should not be necessary as found in json file for each experiment.
  dvar='tas'
  dvar='thetao'
  dvar='tos'

  ybeg=491
  yend=500
  mbeg_first=1
  mend_last=12
  
  line_kwargs='rundir='+rundir+' dvar='+dvar+' ybeg='+str(ybeg)+' yend='+str(yend)+ \
    ' mbeg_first='+str(mbeg_first)+' mend_last='+str(mend_last)+ \
    ' daily_data_layout='+daily_data_layout
  
  if(BATCH):
    os.chmod(script,500)
    
    print('Copying and editing '+prodir+'/'+'qjob.csh to '+rundir+'/'+'qjob.csh')
    ifh=open(prodir+'/'+'qjob.csh')
    ofh=open(rundir+'/'+'qjob.csh','w')
    for i,line in enumerate(ifh):
      line=line.replace('CAFEPP_SCRIPT','./'+script+' '+line_kwargs)
      line=line.replace('RUNDIR',rundir)
      line=line.replace('CONDA_SOURCE','. /short/v14/mac599/anaconda3/etc/profile.d/conda.sh')
      line=line.replace('CONDA_ACTIVATE','conda activate cafepp_27_scipy')
      print(line,file=ofh,end='')
    ifh.close()
    ofh.close()
      
    os.system('qsub '+rundir+'/'+'qjob.csh')
  else:
    print('Current Working Directory=',os.getcwd())
    os.chdir(rundir)
    
    kwargs=dict(x.split('=', 1) for x in line_kwargs.split(' '))
    print('kwargs=',kwargs
         )
    #cafepp_monthly_forecast.main(**kwargs)
      
    cafepp_daily.main(**kwargs)
    
    #cafepp_daily_assimilation_year_month.main(rundir)
    
print('END')

## information common to processing all runs...

In [ ]:
raise Exception('STOP!')

In [ ]:
print('BEGIN.')

import math
from array import array

#remember on 1x1 degree latxlon grid.

indices_nino=['nino34','nino3','nino4','nino1+2'] # 170W-120W, 150W-90W, 160E-150W, 90W-80W:270-280 (10S-0 latitude, others all 5S to 5N)
indices_label=['Ni$\~{n}$o3.4','Ni$\~{n}$o3','Ni$\~{n}$o4','Ni$\~{n}$o1+2']
  
#if(cafe_grid=='gr2'):
#elif(cafe_grid=='gn'):

indices_i_gr2,indices_j_gr2=[[190,239],[210,269],[150,209],[270,280]],[[85,94],[85,94],[85,94],[80,89]] #check this, whether I need +1 also...what about fractional cells?
indices_i_gn,indices_j_gn=[[110,159],[130,189],[80,129],[190,199]],[[122,151],[122,151],[122,151],[107,136]] #check this, whether I need +1 also...what about fractional cells?

#indices_select=array('i',[0,1,2,3])
#indices_select=[np.int(0)]
#indices_select=range(3)
#print(type(indices_select))
#print('indices_select=',indices_select)

#used for speeding things up, either can choose a single indice or all - don't know why can't use : in eval for e.g.
sss='-1' #last
sss='0' #first
#sss='ALL'


#raise Exception('STOP!')

if(sss!='ALL'):
  indices_nino=[indices_nino[eval(sss)]]
  indices_label=[indices_label[eval(sss)]]
  indices_i_gr2=[indices_i_gr2[eval(sss)]]
  indices_j_gr2=[indices_j_gr2[eval(sss)]]
  indices_i_gn=[indices_i_gn[eval(sss)]]
  indices_j_gn=[indices_j_gn[eval(sss)]]

print('indices_nino=',indices_nino)

print('indices_i_gr2=',indices_i_gr2)

nindices_nino=len(indices_nino)

#else:
#  raise Exception('STOP!')
#raise Exception('STOP!')
#del(get_timestamp_number)
#from decadal_diag import get_timestamp_number

rad = 4.0*math.atan(1.0)/180.0
#print('rad=',rad)

#%who

print('END.')

# Now to plot data, this will depend on success of previous steps producing necessary outputs with cafepp(_daily).

I've also updated with an option to read in a "raw" monthly file and to overlay it. Note that here it includes a partial final year whereas the daily one doesn't.

In [ ]:
print('BEGIN.')

import math

os.chdir(rundir)

cafe_grid='gr2'
cafe_grid='gn'

files_string='CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r1i1p2f1/Oday/'+dvar+'/'+cafe_grid+'/v20171025/'+dvar+'_Oday_historical_CAFE-1-0_r1i1p2f1_'+cafe_grid+'_????????-????????.nc'
files=glob.glob(files_string)
files=sorted(restrict_input_files(files,28,31))

#print('files=',files)
#for i,file in enumerate(files):
#  print('i,file=',i,file)
#raise Exception('STOP!')
  
ifhN=netCDF4.MFDataset(files)
ifh0=netCDF4.Dataset(files[0])

time=ifhN.variables['time']
#lat=ifh0.variables['lat']
#lon=ifh0.variables['lon']

lat=ifh0.variables['latitude'][:,0]
lon=ifh0.variables['longitude'][0,:]

ntime=len(time)
#print('lat=',lat)

ifh0.close()

#print('rad=',rad)
clat=np.cos(lat[:]*rad)
#print('clat=',clat)

#print('lat=',lat[85:95])
#print('lon=',lon[190:240])

nino=ma.zeros((nindices_nino,ntime),dtype=float)

#tos_data=(ifhN.variables['tos'])
#tos_data=np.ma.masked_array(tos_data,np.isnan(tos_data))

for k,indice in enumerate(indices_nino):
  print('indice=',indice)
  #if(indice=='nino1+2'):
  imin,imax=indices_i_gn[indices_nino.index(indice)][0],indices_i_gn[indices_nino.index(indice)][1]
  jmin,jmax=indices_j_gn[indices_nino.index(indice)][0],indices_j_gn[indices_nino.index(indice)][1]
  nino_box=ifhN.variables[dvar][:,jmin:jmax+1,imin:imax+1]
  nino_box=np.ma.masked_array(nino_box,np.isnan(nino_box))
  #print('lat[jmin:jmax]=',lat[jmin:jmax+1])
  #print('lon[imin:imax]=',lon[imin:imax+1])
    #print(ifhN.variables['tos'][:,jmin:jmax,imin:imax])
    #print('nino_box=',nino_box)
    #raise Exception('STOP!')
    #np.ma.masked_array(data,np.isnan(dat))
  nino[k,:]=np.average(np.average(nino_box,axis=1,weights=clat[jmin:jmax+1]),axis=1)
  #nino[k,:]=np.average(np.average(ifhN.variables['tos'][:,jmin:jmax+1,imin:imax+1],axis=1,weights=clat[jmin:jmax+1]),axis=1)

#ifhN.close()

'''
print('time.units=',time.units)
print('time.calendar=',time.calendar)
print('time=',time)
print('time[:]=',time[:])
'''

date_time_stamp=netCDF4.num2date(time[:],time.units,time.calendar)

#print('date_time_stamp=',date_time_stamp)

'''
print('date_time_stamp=',date_time_stamp)
num_stamp=netCDF4.date2num(date_time_stamp,time.units,time.calendar)
print('num_stamp=',num_stamp)
print('year_fraction=',year_fraction)
'''
year_fraction=fractional_year_from_num2date(date_time_stamp,time.calendar)

#print('year_fraction=',year_fraction)
#print('nino[0,:]=',nino[0,:])

%matplotlib inline

print('nino.shape=',nino.shape)
print("indices_nino.index('nino34')=",indices_nino.index('nino34'))

fix,ax=plt.subplots()
for k,indice in enumerate(indices_nino):
  ax.plot(year_fraction,nino[indices_nino.index(indice),:],label=indice)
legend=ax.legend(loc='lower right',shadow=False,fontsize='x-large')
plt.title('Daily values')
#plt.plot(num_stamp,nino34[:])
plt.xlabel('Year')
plt.ylabel('$^o$C')
plt.show()

print('END')

Work on code to determine how to calculate monthly averages from daily data.

Thinking of 4 options, assuming that the time-series is assiminuous and increasing:
1. exclude any partial months at beginning and end of time-series
2. include partial months at beginning of time-series
3. include partial months at end of time-series
4. include partial months at beginning and end of time-series.

This includes a function and a set of tests.

In [ ]:
print('BEGIN')

import inspect
__file__='jupyter_notebook' #this can be deleted when written to a python script and loaded as module.

ReadPlotMonthly=True #read in the monthly cmorised file and plot it over that calculated from daily inputs to check consistency.
#ReadPlotMonthly=False

if(ReadPlotMonthly):
  cafe_grid='gn'
  ifil_raw='CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r1i1p2f1/Omon/tos/'+cafe_grid+'/v20171025/tos_Omon_historical_CAFE-1-0_r1i1p2f1_'+cafe_grid+'_200201-201606.nc'
  ifh_raw=netCDF4.Dataset(ifil_raw)
  time_raw=ifh_raw.variables['time']
  ntime_raw=len(time_raw)
  lat_raw=ifh_raw.variables['latitude'][:,0]
  lon_raw=ifh_raw.variables['longitude'][0,:]
  clat_raw=np.cos(lat_raw[:]*rad)
  
  nino_monthly_raw=ma.zeros((nindices_nino,ntime_raw),dtype=float)

  for k,indice in enumerate(indices_nino):
    imin,imax=indices_i_gn[indices_nino.index(indice)][0],indices_i_gn[indices_nino.index(indice)][1]
    jmin,jmax=indices_j_gn[indices_nino.index(indice)][0],indices_j_gn[indices_nino.index(indice)][1]
    nino_monthly_box=ifh_raw.variables[dvar][:,jmin:jmax+1,imin:imax+1]
    nino_monthly_box=np.ma.masked_array(nino_monthly_box,np.isnan(nino_monthly_box))
    nino_monthly_raw[k,:]=np.average(np.average(nino_monthly_box,axis=1,weights=clat_raw[jmin:jmax+1]),axis=1) #need to add in area weighting strictly
  date_time_stamp_raw=netCDF4.num2date(time_raw[:],time_raw.units,time_raw.calendar)
  year_fraction_monthly_raw=fractional_year_from_num2date(date_time_stamp_raw,time_raw.calendar)
  
#  %matplotlib inline
#  plt.title('Monthly values from monthly inputs')
#  plt.plot(year_fraction_monthly_raw,nino34_monthly_raw[:])
#  plt.xlabel('Year')
#  plt.ylabel('Ni$\~{n}$o34 ($^o$C)')
#  plt.show()
  
#print('len(date_time_stamp)=',len(date_time_stamp))

#num_stamp=netCDF4.date2num(date_time_stamp,time.units,time.calendar)

#print('type(num_stamp)=',type(num_stamp))
#print('num_stamp=',num_stamp)

#j=netCDF4.num2date(num_stamp,time.units,time.calendar)

#print('j=',j)

#raise Exception('STOP!')
  
#try some tests:

what_to_keep='all_times'
#what_to_keep='all_but_first'
#what_to_keep='all_but_first'
#what_to_keep='all_but_first'

if(what_to_keep=='all_times'):
  #all times kept:
  date_time_stamp_new=date_time_stamp
  nino_new=nino
  year_fraction_new=year_fraction
  
elif(what_to_keep=='all_but_first'):
  #remove first day:
  date_time_stamp_new=date_time_stamp[1::]
  nino_new=nino[:,1::]
  year_fraction_new=year_fraction[1::]
  
elif(what_to_keep=='all_but_first'):
  #remove last day:
  date_time_stamp_new=date_time_stamp[0:-1]
  nino_new=nino[:,0:-1]
  year_fraction_new=year_fraction[0:-1]

elif(what_to_keep=='all_but_first'):
  #remove first and last day:
  date_time_stamp_new=date_time_stamp[1:-1]
  nino_new=nino[:,1:-1]
  year_fraction_new=year_fraction[1:-1]
  
else:
  raise Exception('That what_to_keep option doesnt exist.')

print('len(date_time_stamp_new)=',len(date_time_stamp_new))
#print('date_time_stamp_new=',date_time_stamp_new)

daily_month_indice_beg,daily_month_indice_end,daily_year_beg,daily_year_end,daily_month_beg,daily_month_end,daily_day_beg,daily_day_end,beg_month_partial,end_month_partial = \
  generate_daily_month_indices(date_time_stamp_new,time.units,time.calendar,1)

#print('beg_month_partial,end_month_partial=',beg_month_partial,end_month_partial)
#print('len(daily_month_indice_beg)=',len(daily_month_indice_beg))
#print('daily_month_indice_beg=',daily_month_indice_beg)

#print('daily_month_indice_end=',daily_month_indice_end)
#print('daily_year_beg=',daily_year_beg)
#print('daily_year_end=',daily_year_end)
#print('daily_month_beg=',daily_month_beg)
#print('daily_month_end=',daily_month_end)
#print('daily_day_beg=',daily_day_beg)
#print('daily_day_end=',daily_day_end)
  
#raise Exception('STOP!')

nino_monthly=ma.zeros((nindices_nino,len(daily_month_indice_beg)),dtype=float)

#print('nino_monthly.shape=',nino_monthly.shape)
#print('nino_new.shape=',nino_new.shape)

#raise Exception('STOP!')

for k,indice in enumerate(indices_nino):
  for month in range(0,len(daily_month_indice_beg)):
    nino_monthly[k,month]=np.average(nino_new[k,daily_month_indice_beg[month]:daily_month_indice_end[month]+1])

#print(nino_monthly[:,0])
#raise Exception('STOP!')

year_fraction_monthly=ma.zeros(len(daily_month_indice_beg))
for month in range(0,len(daily_month_indice_beg)):
  year_fraction_monthly[month]=np.average(year_fraction_new[daily_month_indice_beg[month]:daily_month_indice_end[month]+1])

num_stamp_new=netCDF4.date2num(date_time_stamp_new,time.units,time.calendar)

num_stamp_monthly=np.zeros(len(daily_month_indice_beg))
print('num_stamp_monthly.shape=',num_stamp_monthly.shape)

for month in range(0,len(daily_month_indice_beg)):
  num_stamp_monthly[month]=np.average(num_stamp_new[daily_month_indice_beg[month]:daily_month_indice_end[month]+1])

#print('num_stamp_monthly_new=',num_stamp_monthly_new)

date_time_stamp_monthly=netCDF4.num2date(num_stamp_monthly,time.units,time.calendar)

#print('date_time_stamp_monthly_new',date_time_stamp_monthly_new)

#raise Exception('STOP!')
  
EndOption=1 #use months with no days missing at begin and end months (i.e. discard partial months if they exist).
#EndOption=2 #use months with days missing at both begin and end months (i.e. don't discard partial months if they exist @ beg/end).

if(EndOption==1):
  print('Discarding beg&/end month if they exist.')
  if(beg_month_partial or end_month_partial):
    if(beg_month_partial and end_month_partial):
      print('type#1')
      nino_monthly_new=nino_monthly[:,1:-1]
      year_fraction_monthly_new=year_fraction_monthly[1:-1]
      date_time_stamp_monthly_new=date_time_stamp_monthly[1:-1]
      
    elif(not beg_month_partial and end_month_partial):
      print('type#2')
      nino_monthly_new=nino_monthly[:,0:-1]
      year_fraction_monthly_new=year_fraction_monthly[0:-1]
      date_time_stamp_monthly_new=date_time_stamp_monthly[0:-1]

    elif(beg_month_partial and not end_month_partial):
      print('type#3')
      nino_monthly_newnino=nino_monthly[:,1::]
      year_fraction_monthly_new=year_fraction_monthly[1::]
      date_time_stamp_monthly_new=date_time_stamp_monthly[1::]

    else:
      raise SystemExit('Shouldnt get here:'+__file__+' line number: '+str(inspect.stack()[0][2]))
  else:
    print('type#4')
    nino_monthly_new=nino_monthly
    year_fraction_monthly_new=year_fraction_monthly
    date_time_stamp_monthly_new=date_time_stamp_monthly
    
elif(EndOption==2):
  print('Keeping beg/end month or both.')
  nino_monthly_new=nino_monthly
  year_fraction_monthly_new=year_fraction_monthly
  date_time_stamp_monthly_new=date_time_stamp_monthly
  
else:
  raise SystemExit('EndOption can be only 1 or 2:'+__file__+' line number: '+str(inspect.stack()[0][2]))
  
#raise Exception('STOP!')

%matplotlib inline

#print(nino_monthly_new)

#print('year_fraction_monthly_new=',year_fraction_monthly_new)

plot_index='nino34'
#plot_index='nino1+2'
#plot_index='nino4'

fix,ax=plt.subplots()
ax.plot(year_fraction_monthly_new,nino_monthly_new[indices_nino.index(plot_index),:],color='blue',label='mon_from_day')
if(ReadPlotMonthly): ax.plot(year_fraction_monthly_raw,nino_monthly_raw[indices_nino.index(plot_index),:],color='red',label='mon')
legend=ax.legend(loc='lower left',shadow=False,fontsize='large')
plt.title('Monthly values')
plt.xlabel('Year')
plt.ylabel(indices_label[indices_nino.index(plot_index)]($^o$C))
plt.show()

print('END')

Would like to caculate climatology/anomaly taking into account years with missing months.

In [ ]:
print('BEGIN.')

nmy=12

ybeg=date_time_stamp_monthly_new[0].year
yend=date_time_stamp_monthly_new[-1].year

#print('ybeg,yend=',ybeg,yend)

#print('date_time_stamp=',date_time_stamp)

#raise Exception('STOP!')
  
ydiff=yend-ybeg+1

MissingMonths=False

first_month=date_time_stamp_monthly_new[0].month
last_month=date_time_stamp_monthly_new[-1].month

#print('first_month,last_month=',first_month,last_month)
#print('ydiff=',ydiff)

missing_months_beg,missing_months_end=0,0

cyear_beg_skip,cyear_end_skip=0,1

if(first_month!=1):
  missing_months_beg=first_month-1
  cyear_beg_skip=1
  MissingMonths=True

if(last_month!=12):
  missing_months_end=12-last_month
  cyear_end_skip=2
  MissingMonths=True

if(MissingMonths):
  print('There are missing months in the set. '+str(missing_months_beg)+' at beginning and '+str(missing_months_end)+' at end.')
  print('Currently years with missing months are not used in generating long term monthly climatology.')
  print('And missing months will be set to missing in the final time-series.')
  ts_beg,ts_end,ts_avg,dt_beg,dt_end,dt_avg=get_timestamp_number(ybeg,yend,1,12,time.units,time.calendar)
  year_fraction_monthly_full=fractional_year_from_num2date(ts_avg,time.calendar)
  
  nino_monthly_full=ma.masked_all((nindices_nino,ydiff*nmy),dtype=float) #ensure missing months are masked out.
  
  print('nino_monthly_full.shape=',nino_monthly_full.shape)
  
  last_month_index=ydiff*nmy-(12-last_month)
  print('first_month-1,last_month_index=',first_month-1,last_month_index)
  nino_monthly_full[:,first_month-1:last_month_index]=nino_monthly_new
else:
  print('All years have 12 months of non-missing data.')
  
  nino_monthly_full=nino_monthly_new
  year_fraction_monthly_full=year_fraction_monthly_new
  
#print(nino_monthly_full[:,0])

#print('nino_monthly_full.shape=',nino_monthly_full.shape)
#print('nino_monthly_full.shape[1]=',nino_monthly_full.shape[1])

#print('cyear_beg_skip,cyear_end_skip=',cyear_beg_skip,cyear_end_skip)


#raise Exception('STOP!')

#keep all indices for monthly anomaly and 12 monthly climatology, other variables can be temporary.
nino_monthly_anomaly=ma.zeros((nindices_nino,len(daily_month_indice_beg)),dtype=float)
nino_climatology=ma.zeros((nindices_nino,12),dtype=float)

for k,indice in enumerate(indices_nino):
  #print('indice=',indice)
  #print(new_monthly_array_shape(nino34_monthly_full[k,:].shape,ydiff,nmy))
  #raise Exception('STOP!')
  nino_monthly_reshaped=np.reshape(nino_monthly_full[k,:],new_monthly_array_shape(nino_monthly_full[k,:].shape,ydiff,nmy)) #check this works on large multi-dimensional arrays.
  #print('nino_monthly_reshaped.shape=',nino_monthly_reshaped.shape)
  nino_climatology[k]=np.average(nino_monthly_reshaped[cyear_beg_skip:-cyear_end_skip],axis=0) #average over full years only, this could be an option as could use for all years present.
  nino_monthly_climatology=np.expand_dims(nino_climatology[k],0)
  nino_monthly_climatology=np.tile(nino_monthly_climatology,(ydiff,1))
  nino_monthly_climatology_flat=nino_monthly_climatology.flatten()
  nino_monthly_anomaly[k,:]=nino_monthly_full[k,:]-nino_monthly_climatology_flat
  
%matplotlib inline

zero=np.zeros(len(year_fraction_monthly_full))
fix,ax=plt.subplots()
for k,indice in enumerate(indices_nino):
  ax.plot(year_fraction_monthly_full,nino_monthly_anomaly[k,:],label=indice)
plt.plot(year_fraction_monthly_full,zero,color='black')

legend=ax.legend(loc='upper center',shadow=False,fontsize='x-large')
plt.title('Monthly anomalies')
plt.xlabel('Year')
plt.ylabel('$^o$C')
plt.show()

#print('nino_monthly_anomaly[0]=',nino_monthly_anomaly[1])
#print(nino_monthly_anomaly[:,0])

print('END.')

In the following monthly values will be calculated and then plotted. This approach assumes that all months have all days therefore is obsolete
as cell above can deal with partial beg/end months of data.

In [ ]:
'''ReadPlotMonthly=True #read in the monthly cmorised file and plot it over that calculated from daily inputs to check consistency.
#ReadPlotMonthly=False

#way to get it to reload the function without having to reset the kernel:
#del(get_daily_indices_for_monthlyave)
from decadal_diag import get_daily_indices_for_monthlyave

if(ReadPlotMonthly):
  ifil_raw='CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r1i1p2f1/Omon/tos/gn/v20171025/tos_Omon_historical_CAFE-1-0_r1i1p2f1_gn_200201-201606.nc'
  ifh_raw=netCDF4.Dataset(ifil_raw)
  time_raw=ifh_raw.variables['time']
  lat_raw=ifh_raw.variables['latitude'][:,0]
  lon_raw=ifh_raw.variables['longitude'][0,:]
  clat_raw=np.cos(lat_raw[:]*rad)
  nino34_monthly_raw=np.average(np.average(ifh_raw.variables['tos'][:,122:152,110:160],axis=1,weights=clat_raw[122:152]),axis=1) #need to add in area weighting strictly
  date_time_stamp_raw=netCDF4.num2date(time_raw[:],time_raw.units,time_raw.calendar)
  year_fraction_monthly_raw=fractional_year_from_num2date(date_time_stamp_raw,time_raw.calendar)
#  %matplotlib inline
#  plt.title('Monthly values from monthly inputs')
#  plt.plot(year_fraction_monthly_raw,nino34_monthly_raw[:])
#  plt.xlabel('Year')
#  plt.ylabel('Ni$\~{n}$o34 ($^o$C)')
#  plt.show()

  #raise Exception('STOP!')
  
daily_indices=get_daily_indices_for_monthlyave(time,date_time_stamp,time.calendar) #a list

print('daily_indices=',daily_indices)

nino34_monthly=np.zeros(len(daily_indices))
for month in range(0,len(daily_indices)):
  nino34_monthly[month]=np.average(nino34[daily_indices[month]])

year_fraction_monthly=np.zeros(len(daily_indices))
for month in range(0,len(daily_indices)):
  year_fraction_monthly[month]=np.average(year_fraction[daily_indices[month]])
  
%matplotlib inline

plt.plot(year_fraction_monthly,nino34_monthly,color='blue')
if(ReadPlotMonthly): plt.plot(year_fraction_monthly_raw,nino34_monthly_raw,color='red')
plt.title('Monthly values')
plt.xlabel('Year')
plt.ylabel('Ni$\~{n}$o34 ($^o$C)')
plt.show()
'''

In [ ]:
ybeg=date_time_stamp[0].year
yend=date_time_stamp[-1].year
ydiff=yend-ybeg+1
nmy=12

nino34_monthly_reshaped=np.reshape(nino34_monthly,new_monthly_array_shape(nino34_monthly.shape,ydiff,nmy))
climatology=np.average(nino34_monthly_reshaped,axis=0)
nino34_monthly_climatology=np.expand_dims(climatology,0)
nino34_monthly_climatology=np.tile(nino34_monthly_climatology,(ydiff,1))
nino34_monthly_climatology_flat=nino34_monthly_climatology.flatten()
nino34_monthly_anomaly=nino34_monthly-nino34_monthly_climatology_flat

%matplotlib inline

zero=np.zeros(len(daily_indices))
plt.plot(year_fraction_monthly,nino34_monthly_anomaly)
plt.plot(year_fraction_monthly,zero)

plt.title('Monthly anomalies')
plt.xlabel('Year')
plt.ylabel('Ni$\~{n}$o34 ($^o$C)')
plt.show()